In [1]:
import scipy.io
import tensorflow as tf
import numpy as np
from tensorflow import  keras
from keras.callbacks import TensorBoard
from sklearn.preprocessing import MinMaxScaler,StandardScaler,scale
from skimage.measure import block_reduce
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout

In [2]:
def load_data():
    als = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/ALS.mat',)["data"].transpose()  #(262134, 321)
    normal = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/normal.mat')['data'].transpose() # (262134, 270) 
    myopathy = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/myopathie.mat')['data'].transpose() #(262134, 315)
    return als,normal,myopathy

In [3]:

def scale_data_minmax(data):
#type of scaler that scales the minimum and maximum values to be 0 and 1 respectively. 
    scaler = MinMaxScaler()
#fit_transform() then it will calculate the mean(μ) and standard deviation(σ) of the feature F at a time it will transform the data points of the feature F.    
    normalized_data =scaler.fit_transform(data)


    return normalized_data

In [4]:
def scale_data_standard(data):
    scaled_data = scale(data,axis=1)
    return scaled_data

In [5]:
def gen_tensors_list(data,dim):
    dataset  = list()
    for i in range (len(data)):
        arg = tf.convert_to_tensor(data[i], dtype=tf.float32)
        arg = tf.reshape(arg,dim)
        dataset.append(arg)

    return dataset

In [6]:
def windowing_fun(data,window_size=1000,overlap=100):
    windowed_data = [data[j][i : i + window_size] for j in range(0,len(data)) for i in range(0, len(data[j]), window_size-overlap)]
    final_data = [windowed_data[i] for i in range(len(windowed_data)) if len(windowed_data[i]) == window_size ]
    return final_data

In [7]:
def load_file_names():
    als = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/ALS.mat',)["files"]  #(262134, 321)
    normal = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/normal.mat')['files'] # (262134, 270) 
    myopathy = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/myopathie.mat')['files'] #(262134, 315)
    return als,normal,myopathy

In [8]:
def generate_dataframe(data,files):
    df = pd.DataFrame([], columns=['type_signal',"num_personne","muscle","num_enregistrement","signal"])
    windowed_data = [{"signal": data[j],"type_signal":files[0][j][0][5],"num_personne": files[0][j][0][6:8],"muscle": files[0][j][0][8:10],"num_enregistrement":files[0][j][0][10:12] } for j in range(0,len(data)) ]
    final_data = pd.DataFrame(windowed_data)
    #df =df.append(final_data, ignore_index=True)
    #df =pd.concat([df, final_data])
    df = pd.concat([df, final_data], ignore_index=True)
    return df

In [9]:
als, normal , myopathy = load_data()

In [10]:
alsfiles,normalfiles,myopathyfiles = load_file_names()

In [11]:
df = generate_dataframe(np.vstack((normal,als,myopathy)),np.concatenate((normalfiles,alsfiles,myopathyfiles),axis=1))

In [12]:
def f(row):
    return {"C":0,"A":1,"M":2}[row["type_signal"]]

df["num_class"] =df.apply(f, axis=1) 

In [13]:
df = df.query("muscle=='BB'")

In [14]:
X = df['signal'].tolist()
Y = df['num_class'].tolist()

In [15]:
X=X[170:]
Y=Y[170:]

In [16]:
del df

In [17]:
random_noise_1 = np.random.normal(0,1,len(X[0]))
random_noise_2 = np.random.normal(0,1,len(X[0]))
random_noise_3 = np.random.normal(0,1,len(X[0]))
random_noise_4 = np.random.normal(0,1,len(X[0]))
np.shape(random_noise_1)

(262134,)

In [18]:
data = [*X]
#data, np.shape(data)

In [19]:
data  += list(map(lambda x : x + random_noise_1, X))
data  += list(map(lambda x : x + random_noise_2, X))
data  += list(map(lambda x : x + random_noise_3, X))
#data  += list(map(lambda x : x + random_noise_4, X))
print(np.asarray(data).shape)
#print(data)

(1220, 262134)


In [20]:
del  als,normal,myopathy,alsfiles,normalfiles,myopathyfiles 

In [21]:
data = scale_data_standard(data)

In [22]:
last_index_normal = len(Y) - 1 - Y[::-1].index(0)

In [23]:
last_index_als = len(Y) - 1 - Y[::-1].index(1)

In [24]:
last_index_myopathy = len(Y) - 1 - Y[::-1].index(2)

In [25]:
windowed_normal = len(windowing_fun(X[:last_index_normal+1],10000))
windowed_als = len(windowing_fun(X[last_index_normal+1:last_index_als+1],10000))
windowed_myopathy = len(windowing_fun(X[last_index_als+1:],10000))

In [26]:
Y = [0]*(windowed_normal)+[1]*(windowed_als)+[2]*(windowed_myopathy)

In [27]:
Y  = Y*4

In [28]:
X = windowing_fun(data,10000)

In [29]:
del data

In [30]:
X = scale_data_standard(X)

In [31]:
tensors_list = gen_tensors_list(X,(1,10000))

In [32]:
X_train, X_test, y_train, y_test = train_test_split( tensors_list, Y, test_size=0.04,random_state=20)
np.shape(X_train), np.shape(X_test)

((30451, 1, 10000), (1269, 1, 10000))

In [33]:
model = Sequential([
    #Conv2D(#of filters, filters size, activation function)
    #in keras for the first layer we always need to mention the input shape
    Conv1D(64, 3, activation = 'relu', padding ="same", input_shape = (1,10000)), 
    
    #MaxPooling2D(filter size stride and pad can also be added)
    MaxPooling1D(2, strides = 2, padding ="same"),
    
    #LAYER2
    Conv1D(32, 3, activation = 'relu', padding ="same"),
    MaxPooling1D(2, strides = 2, padding ="same"),
    
    #FLATTEN
    Flatten(),
    
    #FC Layers:
    
    #LAYER3
    #Dense : fully connected  Dense(#neurons, activation function )
    Dense(64, activation = 'relu'),
    Dropout(0.6),
    
    #LAYER4
    Dense(3, activation = 'softmax'),
    
    
])


In [34]:
logdir = "drive/MyDrive/PFE_TENSORBOARD_DATA/augmentation0/CNN1D/logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
callbacks = [ tf.keras.callbacks.TensorBoard(log_dir=logdir),tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy',patience =60 ,min_delta=1e-3)]


In [35]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [36]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_text = np.asarray(X_test)
y_test = np.asarray(y_test)

In [37]:
model.fit(X_train, y_train, epochs =60, batch_size = 128)

Epoch 1/60
238/238 [==============================] - 35s 131ms/step - loss: 1.0439 - accuracy: 0.4585
Epoch 2/60
238/238 [==============================] - 31s 132ms/step - loss: 0.7352 - accuracy: 0.6769
Epoch 3/60
238/238 [==============================] - 31s 129ms/step - loss: 0.4515 - accuracy: 0.8197
Epoch 4/60
238/238 [==============================] - 36s 151ms/step - loss: 0.2851 - accuracy: 0.8907
Epoch 5/60
238/238 [==============================] - 42s 176ms/step - loss: 0.2153 - accuracy: 0.9204
Epoch 6/60
238/238 [==============================] - 40s 167ms/step - loss: 0.1505 - accuracy: 0.9458
Epoch 7/60
238/238 [==============================] - 43s 179ms/step - loss: 0.1144 - accuracy: 0.9590
Epoch 8/60
238/238 [==============================] - 41s 171ms/step - loss: 0.1142 - accuracy: 0.9595
Epoch 9/60
238/238 [==============================] - 37s 155ms/step - loss: 0.0990 - accuracy: 0.9657
Epoch 10/60
238/238 [==============================] - 35s 148ms/step - l

In [38]:
score = model.evaluate(np.asarray(X_test),np.asarray (y_test))

40/40 [==============================] - 1s 4ms/step - loss: 0.1075 - accuracy: 0.9890


In [54]:
tensorboard --logdir=/tmp  --port=6007

In [39]:
%reload_ext tensorboard

In [40]:
%tensorboard --logdir drive/MyDrive/PFE_TENSORBOARD_DATA/augmentation0/CNN1D/logs/scalars/

Reusing TensorBoard on port 6006 (pid 12412), started 3 days, 4:31:27 ago. (Use '!kill 12412' to kill it.)

In [41]:
logdir = "drive/MyDrive/PFE_TENSORBOARD_DATA/augmentation0/CNN1D/logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
callbacks = [ tf.keras.callbacks.TensorBoard(log_dir=logdir),tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy',patience =60 ,min_delta=1e-3)]

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
history = model.fit(
    
    x = np.asarray(X_train)  ,
    y= np.asarray(y_train) ,
    validation_split=0.1,
    epochs=50,
    batch_size=64,
    callbacks=callbacks

    )

Epoch 1/50
429/429 [==============================] - 29s 64ms/step - loss: 0.0531 - accuracy: 0.9858 - val_loss: 0.0208 - val_accuracy: 0.9921
Epoch 2/50
429/429 [==============================] - 27s 63ms/step - loss: 0.0227 - accuracy: 0.9925 - val_loss: 0.0270 - val_accuracy: 0.9895
Epoch 3/50
429/429 [==============================] - 28s 66ms/step - loss: 0.0235 - accuracy: 0.9933 - val_loss: 0.0238 - val_accuracy: 0.9908
Epoch 4/50
429/429 [==============================] - 26s 62ms/step - loss: 0.0375 - accuracy: 0.9898 - val_loss: 0.0463 - val_accuracy: 0.9846
Epoch 5/50
429/429 [==============================] - 27s 63ms/step - loss: 0.0329 - accuracy: 0.9900 - val_loss: 0.0212 - val_accuracy: 0.9911
Epoch 6/50
429/429 [==============================] - 28s 66ms/step - loss: 0.0306 - accuracy: 0.9907 - val_loss: 0.0397 - val_accuracy: 0.9865
Epoch 7/50
429/429 [==============================] - 31s 72ms/step - loss: 0.0253 - accuracy: 0.9920 - val_loss: 0.0388 - val_accuracy:

429/429 [==============================] - 28s 65ms/step - loss: 0.0369 - accuracy: 0.9896 - val_loss: 0.0173 - val_accuracy: 0.9911
Epoch 22/50
429/429 [==============================] - 28s 65ms/step - loss: 0.0173 - accuracy: 0.9935 - val_loss: 0.0085 - val_accuracy: 0.9951
Epoch 23/50
429/429 [==============================] - 26s 60ms/step - loss: 0.0262 - accuracy: 0.9923 - val_loss: 0.0555 - val_accuracy: 0.9836
Epoch 24/50
429/429 [==============================] - 25s 59ms/step - loss: 0.0404 - accuracy: 0.9874 - val_loss: 0.0200 - val_accuracy: 0.9915
Epoch 25/50
429/429 [==============================] - 25s 59ms/step - loss: 0.0149 - accuracy: 0.9952 - val_loss: 0.0151 - val_accuracy: 0.9931
Epoch 26/50
429/429 [==============================] - 25s 59ms/step - loss: 0.0120 - accuracy: 0.9957 - val_loss: 0.0124 - val_accuracy: 0.9938
Epoch 27/50
429/429 [==============================] - 26s 60ms/step - loss: 0.0269 - accuracy: 0.9925 - val_loss: 0.0718 - val_accuracy: 0.97

429/429 [==============================] - 25s 59ms/step - loss: 0.0110 - accuracy: 0.9965 - val_loss: 0.0470 - val_accuracy: 0.9882
Epoch 43/50
429/429 [==============================] - 25s 58ms/step - loss: 0.0351 - accuracy: 0.9907 - val_loss: 0.0421 - val_accuracy: 0.9872
Epoch 44/50
429/429 [==============================] - 25s 59ms/step - loss: 0.0185 - accuracy: 0.9937 - val_loss: 0.0301 - val_accuracy: 0.9931
Epoch 45/50
429/429 [==============================] - 25s 59ms/step - loss: 0.0231 - accuracy: 0.9935 - val_loss: 0.0935 - val_accuracy: 0.9836
Epoch 46/50
429/429 [==============================] - 25s 58ms/step - loss: 0.0349 - accuracy: 0.9901 - val_loss: 0.0261 - val_accuracy: 0.9924
Epoch 47/50
429/429 [==============================] - 25s 57ms/step - loss: 0.0124 - accuracy: 0.9956 - val_loss: 0.0232 - val_accuracy: 0.9947
Epoch 48/50
429/429 [==============================] - 25s 57ms/step - loss: 0.0132 - accuracy: 0.9956 - val_loss: 0.0491 - val_accuracy: 0.98

In [42]:
score = model.evaluate(np.asarray(X_test),np.asarray (y_test) ) 

40/40 [==============================] - 0s 4ms/step - loss: 0.1123 - accuracy: 0.9858


In [43]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [44]:
%tensorboard --logdir drive/MyDrive/PFE_TENSORBOARD_DATA/augmentation0/CNN1D/logs/scalars/

Reusing TensorBoard on port 6006 (pid 12412), started 3 days, 4:53:53 ago. (Use '!kill 12412' to kill it.)

In [45]:
model.save("/content/drive/MyDrive/PFE_RESULTS_DATA/augmentation0/scaling/CNN1D/CNN1D.h5")

In [46]:
import json

with open('/content/drive/MyDrive/PFE_RESULTS_DATA/augmentation0/scaling/CNN1D/CNN1D_score.json', 'w+') as file:

    json.dump(score, file)

In [47]:
model= tf.keras.models.load_model("/content/drive/MyDrive/PFE_RESULTS_DATA/augmentation0/scaling/CNN1D/CNN1D.h5")

In [48]:
tf.keras.utils.plot_model(
    model,
    show_shapes=True
    )

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [49]:
y_pred = model.predict(np.asarray(X_test))

40/40 [==============================] - 0s 4ms/step


In [50]:
y_predicted_labels = [{0:"Normal",1:"Neuropathie",2:"Myopathy"}[i] for i in np.argmax(y_pred,axis=1)]

In [51]:
y_true_labels = [{0:"Normal",1:"Neuropathie",2:"Myopathy"}[i] for i in y_test]

In [52]:
import matplotlib.pyplot as plt
import seaborn as sns

con_mat = tf.math.confusion_matrix(labels=y_test, predictions=np.argmax(y_pred,axis=1)).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = ["Normal","Neuropathie"], 
                     columns = ["Normal","Neuropathie"])

figure = plt.figure(figsize=(5,5))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()


ValueError: Shape of passed values is (3, 3), indices imply (2, 2)